# Expert Knowledge Worker [Dumb Way]

A question answering agent for a specific industry.

- The agent needs to be accurate.
- The solution has to be low cost.
- The solution will use RAG (Retrieval Augmented Generation).
- A first approach will be brute-force.

In [ ]:
import os
import glob
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    print("OPENAI_API_KEY not loaded.")
    sys.exit(1)

In [ ]:
openai = OpenAI()

MODEL = "gpt-4o-mini"

In [ ]:
context = {}

employees = glob.glob("knowledge-base/employees/*")

employees

In [ ]:
for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee, "r") as f:
        doc = f.read()
    context[name] = doc

context.keys()

In [ ]:
system_prompt = "You are an expert in answering accurate questions about an Insurance Tech company. Give brief, accurate answers and if you have no knowledge about something do not make up anything and inform the user of your absent knowledge instead."

In [ ]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        # This match is very dumb of course
        if context_title in message:
            relevant_context.append(context_details)
    return relevant_context

In [ ]:
get_relevant_context("Who is Emily Tran")

In [ ]:
def add_relevant_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\nThe following additional context might be relevant in answering this question. If it seems irrelevant please do not make up things and inform the user of no knowledge on the topic. The context follows:\n"
        for r in relevant_context:
            message += r + "\n"
    return message

In [ ]:
add_relevant_context("Who is Emily Tran")

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history
    message = add_relevant_context(message)
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
view = gr.ChatInterface(chat, type="messages").launch()